# Prototype Bemchmark
For each record to be scored, the "county" specific model is read from external storage.  No model caching in memory.

`model_load_time` (seconds) is the time required to read a model file from external storage and deserialize the model object.
`model_score_time` (seconds) is the time to score one record.

## Metrics reported in this notebook are from synthetic data and **have not** been calibrated to representative dataset or model sizes.

## Notebook run-time enviornment
* **Hardware:** MacBook Pro Intel(2019), 16GB RAM, 1TB SSD drive
* **OS:** MacOS 11.6.1
* **Docker:** Docker for Desktop 4.2.0 (Mac)
* **Docker Image:** Base image: `jupyter/datascience-notebook:lab-3.2.5` with ONNX packages added

In [1]:
import os
import shutil
import pandas as pd
import numpy as np
import onnxruntime as rt
import pickle

## Change to project root directory

In [2]:
# required to allow for import of project speccific utility functions
os.chdir('..')

In [3]:
# import project specific utiity functions
from utils.utils import BenchmarkDriver, load_config

In [4]:
# get configuration parameters
config = load_config('./config.yaml')
config

{'data_dir': '/Users/jim/Desktop/onnx_sandbox/data',
 'models_dir': '/Users/jim/Desktop/onnx_sandbox/models',
 'number_records': 100000,
 'number_features': 20,
 'number_informative': 14,
 'fraction_for_test': 0.2,
 'number_counties': 20,
 'random_seed': 123}

In [5]:
TEST_DATA = os.path.join(config['data_dir'],'benchmark', 'test.parquet')
PERFORMANCE_DATA_DIR = os.path.join(config['data_dir'],'performance')
MODELS_DIR = os.path.join(config['models_dir'], 'benchmark')


In [6]:
# setup directory to collect performance data
shutil.rmtree(PERFORMANCE_DATA_DIR, ignore_errors=True)
os.makedirs(PERFORMANCE_DATA_DIR, exist_ok=True)

## Get some test data

In [7]:
test_df = pd.read_parquet(TEST_DATA)
test_df = test_df.sample(n=20, random_state=config['random_seed'])
test_df.reset_index(inplace=True)
test_df.shape

(20, 23)

In [8]:
test_df[['index','county', 'y']]

,index,county,y
0,12136,cnty0015,-31.640778
1,16812,cnty0004,123.967995
2,10072,cnty0011,-59.197975
3,5850,cnty0010,-154.407074
4,4320,cnty0014,5.997156
5,4429,cnty0003,37.982521
6,11744,cnty0011,-19.720573
7,12106,cnty0011,271.501892
8,4230,cnty0019,140.779358
9,16858,cnty0002,45.374973


## Sklearn test

In [9]:
%%time
sklearn_driver = BenchmarkDriver(
    model_type='sklearn',
    models_dir=MODELS_DIR,
    performance_fp=os.path.join(PERFORMANCE_DATA_DIR, 'sklearn_benchmark.csv'),
    test_scenario='county-level'
)

# iterate over each row and collect run-time performance statistics
for idx, row in test_df.iterrows():
    one_record = pd.DataFrame(row).T
    sklearn_driver.score_one_record(row['county'], row['index'],
            one_record.drop(['index', 'county', 'y'], axis='columns'))
    
sklearn_driver.close_performance_data()

CPU times: user 6.68 s, sys: 508 ms, total: 7.18 s
Wall time: 7.14 s


In [10]:
# display collected performance metrics
sklearn_metrics_df = pd.read_csv(os.path.join(PERFORMANCE_DATA_DIR, 'sklearn_benchmark.csv'))
sklearn_metrics_df

,scenario,record_id,model_type,county_id,model_memory_size_mb,model_load_time,model_score_time,prediction
0,county-level,12136,sklearn,cnty0015,54.8020,0.036230,0.025819,-116.880853
1,county-level,16812,sklearn,cnty0004,54.8313,0.016464,0.025898,153.039992
2,county-level,10072,sklearn,cnty0011,54.8325,0.011969,0.025482,-30.620640
3,county-level,5850,sklearn,cnty0010,54.8330,0.012246,0.027017,-89.091659
4,county-level,4320,sklearn,cnty0014,54.8337,0.011988,0.024554,79.284982
5,county-level,4429,sklearn,cnty0003,54.8348,0.012158,0.025724,31.401549
6,county-level,11744,sklearn,cnty0011,54.8361,0.011911,0.028842,-5.722466
7,county-level,12106,sklearn,cnty0011,54.8363,0.012282,0.028745,139.301654
8,county-level,4230,sklearn,cnty0019,54.8370,0.012177,0.030107,103.776609
9,county-level,16858,sklearn,cnty0002,54.8374,0.013236,0.029698,72.877299


## ONNX Test

In [11]:
%%time
onnx_driver = BenchmarkDriver(
    model_type='onnx',
    models_dir=MODELS_DIR,
    performance_fp=os.path.join(PERFORMANCE_DATA_DIR, 'onnx_benchmark.csv'),
    test_scenario='county-level'
)

# iterate over each row and collect run-time performance statistics
for idx, row in test_df.iterrows():
    one_record = pd.DataFrame(row).T
    onnx_driver.score_one_record(row['county'], row['index'],
            one_record.drop(['index', 'county', 'y'], axis='columns').astype(np.float32).to_numpy())
    
onnx_driver.close_performance_data()

CPU times: user 17.6 s, sys: 452 ms, total: 18 s
Wall time: 16.9 s


In [12]:
# display collected performance metrics
onnx_metrics_df = pd.read_csv(os.path.join(PERFORMANCE_DATA_DIR, 'onnx_benchmark.csv'))
onnx_metrics_df

,scenario,record_id,model_type,county_id,model_memory_size_mb,model_load_time,model_score_time,prediction
0,county-level,12136,onnx,cnty0015,54.8081,0.572393,0.000271,-116.880859
1,county-level,16812,onnx,cnty0004,54.8091,0.536377,0.000247,153.039993
2,county-level,10072,onnx,cnty0011,54.8128,0.518280,0.000311,-30.620642
3,county-level,5850,onnx,cnty0010,54.8134,0.500949,0.000243,-89.091667
4,county-level,4320,onnx,cnty0014,54.8135,0.495770,0.000258,79.284981
5,county-level,4429,onnx,cnty0003,54.8137,0.504708,0.000299,31.401548
6,county-level,11744,onnx,cnty0011,54.8138,0.499625,0.000321,-5.722466
7,county-level,12106,onnx,cnty0011,54.8140,0.539727,0.000296,139.301636
8,county-level,4230,onnx,cnty0019,54.8142,0.562006,0.000257,103.776588
9,county-level,16858,onnx,cnty0002,54.8143,0.505992,0.000275,72.877312
